In [1]:
import pylab as plt
import pandas as pd
import numpy as np
import sklearn.impute as skin
from sqlalchemy import create_engine
import locale
import os

output_folder=r"C:\Users\berend\Documents\Python_Scripts\vrije_stage\Results_PIM"
hour=72
locale.setlocale(locale.LC_ALL,'fr_FR')

from all_own_functions import cnfl,value_filtering
from All_standard_variables import conv_dict, dtype_dict,all_cols




C:\Users\berend\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
conv_pice={'BaseExcess':cnfl,'FiO2':cnfl,'PaO2':cnfl,'PIM2Score':cnfl,'PIM2Mort':cnfl,'PIM3Score':cnfl,
'PIM3Mort':cnfl,'GlucoseMax':cnfl,'PaO2Min':cnfl,'PCO2Max':cnfl,'PHMax':cnfl,'PHMin':cnfl,'PotassiumMax':cnfl,'PTMax':cnfl,'TemperatureMax':cnfl,'TemperatureMin':cnfl,
'BicarbonateMin':cnfl,'BicarbonateMax':cnfl,'UreaMax':cnfl,'WhiteBloodCountMin':cnfl,'PRISM4Mortality':cnfl,'Age(days)':cnfl}
PIM3_cols=['HospitalNumber','AdmissionDate','AdmissionPupils','Urgency','Ventilated','BaseExcess','SystolicBP', 'FiO2','PaO2','Recovery','RiskDiagnoses','Status']
df_pice=pd.read_csv('scores.csv',converters=conv_pice,header=0,delimiter=';')
df_pice.drop(df_pice.tail(2).index,inplace=True)
df_pice.dropna(axis=0,how='all',inplace=True)
del df_pice['AdmissionDate']
index_names = df_pice[df_pice['HospitalNumber'] == 'RPH'].index
df_pice.drop(index_names, inplace=True)
a=df_pice[df_pice['HospitalNumber'].duplicated(keep=False)]
a=a[a['HospitalNumber'].duplicated(keep='first')]
duplicate_patients=a['HospitalNumber'].tolist()
df_pice=df_pice.set_index('HospitalNumber')
df_pice=df_pice.sort_index()
pimLowRisk = ["Asthma","Bronchiolitis","Croup","ObstructiveSleepApnea","DiabeticKetoacidosis",'SeizureDisorder']
pimHighRisk = ["CerebralHemorrhage","CardiomyopathyOrMyocarditis","HIVPositive","HypoplasticLeftHeartSyndrome","NeurodegenerativeDisorder","NecrotizingEnterocolitis"]
pimVeryHighRisk = ["CardiacArrestInHospital","CardiacArrestPreHospital","SevereCombinedImmuneDeficiency","LeukemiaorLymphoma","BoneMarrowTransplant","LiverFailure"]

df_pice['RiskDiagnoses'] = (df_pice['RiskDiagnoses'].fillna(value='Unknown'))

for i,row in df_pice.iterrows():
    try:
        if any(substring in df_pice.loc[i,'RiskDiagnoses'] for substring in pimVeryHighRisk) == True:
            df_pice.loc[i,'RiskDiagnoses'] = 3
        elif any(substring in df_pice.loc[i,'RiskDiagnoses'] for substring in pimHighRisk)==True :
            df_pice.loc[i,'RiskDiagnoses']= 2
        elif any(substring in df_pice.loc[i,'RiskDiagnoses'] for substring in pimLowRisk)==True:
            df_pice.loc[i,'RiskDiagnoses'] = 1
        else: df_pice.loc[i,'RiskDiagnoses'] = 0
    except TypeError:
        continue



df_pice=df_pice[~df_pice.index.duplicated(keep='last')]

from sklearn.preprocessing import RobustScaler
scaler=RobustScaler()
float_columns_pice = list(df_pice.select_dtypes(include=['float64']).columns)

df_pice_temp=df_pice[float_columns_pice]
df_pice_scaled=pd.DataFrame(scaler.fit_transform(df_pice_temp), columns=float_columns_pice,index=df_pice_temp.index)
print(df_pice_scaled.info())
df_pice[float_columns_pice]=df_pice_scaled


<class 'pandas.core.frame.DataFrame'>
Float64Index: 9688 entries, 219.0 to 526480713.0
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age(days)                 9688 non-null   float64
 1   AdmissionPupils           9360 non-null   float64
 2   SystolicBP                8213 non-null   float64
 3   BaseExcess                4811 non-null   float64
 4   FiO2                      5087 non-null   float64
 5   PaO2                      4296 non-null   float64
 6   PIM2Score                 9688 non-null   float64
 7   PIM2Mort                  9688 non-null   float64
 8   PIM3Score                 9688 non-null   float64
 9   PIM3Mort                  9688 non-null   float64
 10  CreatinineMax             304 non-null    float64
 11  GlucoseMax                959 non-null    float64
 12  HeartRateMax              1157 non-null   float64
 13  MentalStatus              0 non-null      float64


In [3]:

from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from collections import Counter 
lb=LabelEncoder()
df_pice['Status']=lb.fit_transform(df_pice['Status'])
df_com=df_pice
a=list(df_pice.columns)
data_classes=['Death','Alive']
d = dict(zip(data_classes, range(0,1)))
df_pice['Status'].map(d,na_action='ignore')

x=df_com.drop(['Status'],axis=1)
y=df_com['Status']
x.replace(to_replace='NULL', value=np.nan,inplace=True)
print(x.info())

print(x.head())
print(y.value_counts())


c=x.corrwith(y,method='pearson')
d=x.corrwith(y,method='spearman')
relevant_features_pe = c[c>0.05]
relevant_features_sp = d[d>0.05]
print(relevant_features_pe)
x_pe=x[relevant_features_pe.index]
x_sp=x[relevant_features_sp.index]


<class 'pandas.core.frame.DataFrame'>
Float64Index: 9688 entries, 219.0 to 526480713.0
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age(days)                 9688 non-null   float64
 1   RiskDiagnoses             9688 non-null   object 
 2   Urgency                   9688 non-null   object 
 3   Recovery                  9688 non-null   object 
 4   Ventilated                9688 non-null   object 
 5   AdmissionPupils           9360 non-null   float64
 6   SystolicBP                8213 non-null   float64
 7   BaseExcess                4811 non-null   float64
 8   FiO2                      5087 non-null   float64
 9   PaO2                      4296 non-null   float64
 10  PIM2Score                 9688 non-null   float64
 11  PIM2Mort                  9688 non-null   float64
 12  PIM3Score                 9688 non-null   float64
 13  PIM3Mort                  9688 non-null   float64


In [4]:
c=x['PIM3Score'].corr(x['PRISM3Score'],method='pearson')
print(c)

0.22509779036863556


In [5]:


from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC,SVR,LinearSVC
from sklearn.metrics import classification_report, confusion_matrix,average_precision_score,f1_score,roc_curve,roc_auc_score,plot_confusion_matrix
from sklearn.calibration import calibration_curve
import sklearn.metrics as metrics
from all_own_functions import f_importances
import os


def machine_learning_function(x_train,x_test,y_train,y_test,model,wrapper=0):
    float_columns=list(x_train.select_dtypes(include=['float64']).columns)
    int_columns=list(x_train.select_dtypes(include=['int32']).columns)
    cat_list=list(x_train.select_dtypes(include=['object']).columns)
    
    if 'Label' in float_columns: float_columns.remove('Label')

    float_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent'))])
    int_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='constant',fill_value=0))])
    cat_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='constant',fill_value='Unknown')),('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocess = ColumnTransformer(transformers=[('float',float_transformer,float_columns),('int',int_transformer,int_columns),('cat',cat_transformer,cat_list)],                       remainder='passthrough')

    if wrapper == 1:
        clf = Pipeline(steps=[('preprocessor',preprocess),('Feature_selection',SelectFromModel(LinearSVC(max_iter=5000))),('classifier',(model))])
    elif wrapper == 2:
        clf = Pipeline(steps=[('preprocessor',preprocess),('Feature_selection',SelectFromModel(DecisionTreeClassifier())),('classifier',(model))])
    elif wrapper == 3:
        clf = Pipeline(steps=[('preprocessor',preprocess),('Feature_selection',SelectFromModel(LogisticRegression())),('classifier',(model))])
    else:
        clf = Pipeline(steps=[('preprocessor',preprocess),('classifier',model)])

    clf=clf.fit(x_train,y_train)
    y_pred_clas=clf.predict(x_test)
    # Predict the probabilities, function depends on used classifier

    try:
        y_pred_prob=clf.predict_proba(x_test)
        y_pred_prob=y_pred_prob[:,1]
    except:
        try:
            y_pred_prob=clf.decision_function(x_test)
        except:
            y_pred_prob=y_pred_clas
    
    # failsafe to inpute NaN probabilities with 0
    inds = np.where(np.isnan(y_pred_prob))
    if inds:
        y_pred_prob=np.nan_to_num(y_pred_prob, nan=0)
    return clf, y_pred_clas,y_pred_prob, y_test,x_test

In [6]:

from sklearn.metrics import roc_auc_score
from math import sqrt

def roc_auc_ci(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2*AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC*(1 - AUC) + (N1 - 1)*(Q1 - AUC**2) + (N2 - 1)*(Q2 - AUC**2)) / (N1*N2))
    lower = AUC - 1.96*SE_AUC
    upper = AUC + 1.96*SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower, upper)

In [9]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve,roc_auc_score,brier_score_loss
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
# speciify colours for plot later on
colours=['k-', 'g--', 'r:', 'c-.', 'm-+', 'y-*', 'k-o']

# Stratiefied fold for cros-validation
fold=StratifiedKFold(3)

# Names and classiefiers to be used in the loop
names = [ "Random Forest", "Sigmoid SVM"]

classifiers = [
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1,class_weight={0:0.1,1:0.9}),
    SVC(kernel='sigmoid',probability=True,class_weight={0:0.1,1:0.9}),
    ]

# Names and dataframes used in the loop
data_names=["All_data"]#,'Pearson_Cor',"Wrap_Lin_SVM","Wrap_Dec_Tree"]
x_data=[x['PIM3Score']]#,x_pe, x, x]
auc_pim=0
# split data in train and test data

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=1 / 5,random_state=1)

# calculate ROC and AUC outside of loop
fpr_pim, tpr_pim, _ = roc_curve(y_test,  x_test['PIM3Score'])
auc_pim = roc_auc_score(y_test, x_test['PIM3Score'])

# check if result files do not already exist, updating pdf is not possible
try:
    pdf = PdfPages(os.path.join(output_folder,f"Figures of {hour} hour results.pdf"))
    pdf_ROC = PdfPages(os.path.join(output_folder,f"ROC of {hour} hour results.pdf"))
    pdf_cal = PdfPages(os.path.join(output_folder,f"Cal of {hour} hour results.pdf"))
    pdf_hist = PdfPages(os.path.join(output_folder,f"Hist of {hour} hour results.pdf"))
except PermissionError:
    os.remove(os.path.join(output_folder,f"Figures of {hour} hour results.pdf"))
    os.remove(os.path.join(output_folder,f"ROC of {hour} hour results.pdf"))
    os.remove(os.path.join(output_folder,f"Results_{hour}hours_scores.txt"))
    os.remove(os.path.join(output_folder,f"Cal of {hour} hour results.pdf"))
    os.remove(os.path.join(output_folder,f"Hist of {hour} hour results.pdf"))
    pdf = PdfPages(os.path.join(output_folder,f"Figures of {hour} hour results.pdf"))
    pdf_ROC= PdfPages(os.path.join(output_folder,f"ROC of {hour} hour results.pdf"))
    pdf_cal = PdfPages(os.path.join(output_folder,f"Cal of {hour} hour results.pdf"))
    pdf_hist = PdfPages(os.path.join(output_folder,f"Hist {hour}  hour results.pdf"))

# loop over different data and feature selection techniques
for data_name, xd in zip(data_names, x_data):
    # transform columns based on dataframe used
    #columns = xd.columns.tolist()
    x_train_d = x_train#[columns]
    x_test_d = x_test#[columns]
    
    # Add variable for wrapper based feature selection
    if 'Wrap' in data_name:
        wrapper += 1
    else:
        wrapper=0
    
    # create variables for temp storage lateron
    temp_fpr=dict()
    temp_tpr=dict()
    temp_auc=dict()
    temp_probtrue={}
    temp_probpred={}
    temp_score={}

    # loop over different classifiers
    for name, clf_s in zip(names, classifiers):

        clf,y_pred_clas,y_pred_prob,y_test,x_test_d = machine_learning_function(x_train_d,x_test_d,y_train,y_test,clf_s,wrapper)

        # calculate scoring metrics
        report=classification_report(y_test,y_pred_clas,target_names=['Alive','Death'])
        score=clf.score(x_test_d,y_test)
        average_precision = average_precision_score(y_test, y_pred_prob)
        f1_s=f1_score(y_test, y_pred_clas)

        # write scoring metrics to file
        with open(os.path.join(output_folder,f"Results_{hour}hours_scores.txt"),'a') as file:
            file.write(f"{data_name} with {name} Results for {hour} hours \n\n")
            file.write(f"Classification report \n {report} \n")
            file.write(f"Hold_out_scores {score} \n")
            file.write(f"Average precision score {average_precision} \n")
            file.write(f"F1 score {f1_s} \n\n\n")
        
        # plot confusion matrix
        plot_confusion_matrix(clf,x_test_d,y_test)
        plt.title(f"{data_name} with {name} classifier, Results from {hour} hour data")
        fig=plt.gcf()
        pdf.savefig(fig)
        plt.close(fig)

        # plot ROC with AUC
        fpr, tpr, _ = roc_curve(y_test,  y_pred_prob)
        auc = roc_auc_score(y_test, y_pred_prob)
        plt.plot(fpr,tpr,label=f"{name}, auc={auc}")
        plt.plot(fpr_pim,tpr_pim,label=f"PIM3Score, auc={auc_pim}")
        plt.title(f"{data_name} with {name} classifier, Results from {hour} hour data")
        plt.legend(loc=4)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        fig=plt.gcf()
        pdf.savefig(fig)
        plt.close(fig)

        # temporarly store ROC and AUC per classifier
        temp_fpr.update({f'{name}': fpr})
        temp_tpr.update({f'{name}': tpr})
        temp_auc.update({f'{name}': auc})

         # plot callibration plot with brier_loss score
        
        prob_true, prob_pred = calibration_curve(y_test,  y_pred_prob,normalize=True)
        plt.plot(prob_pred,prob_true,label=f"{name} ")
        plt.title(f"{data_name} with {name} classifier, Results from {hour} hour data")
        plt.legend(loc=4)
        plt.ylabel('Fraction of Positives')
        fig=plt.gcf()
        pdf.savefig(fig)
        plt.close(fig)

        # temporarly store ROC and AUC per classifier
        temp_probtrue.update({f'{name}': prob_true})
        temp_probpred.update({f'{name}': prob_pred})
        #temp_score.update({f'{name}':clf_score})
    
        print(f"{data_name} with {name} classifier, results from {hour} hour data")
        print("Original ROC area: {:0.3f}".format(roc_auc_score(y_test, y_pred_prob)))
        print("Original ROC area, PIM3: {:0.3f}".format(roc_auc_score(y_test, x_test['PIM3Score'])))

        n_bootstraps = 2000
        rng_seed = 42  # control reproducibility
        bootstrapped_scores = []

        rng = np.random.RandomState(rng_seed)
        for i in range(n_bootstraps):
            # bootstrap by sampling with replacement on the prediction indices
            indices = rng.randint(0, len(y_pred_prob), len(y_pred_prob))
            if len(np.unique(y_test.iloc[indices])) < 2:
                # We need at least one positive and one negative sample for ROC AUC
                # to be defined: reject the sample
                continue

            score = roc_auc_score(y_test.iloc[indices], y_pred_prob[indices])
            bootstrapped_scores.append(score)
            #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

        sorted_scores = np.array(bootstrapped_scores)
        sorted_scores.sort()

        # Computing the lower and upper bound of the 90% confidence interval
        # You can change the bounds percentiles to 0.025 and 0.975 to get
        # a 95% confidence interval instead.
        confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
        confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
        print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
            confidence_lower, confidence_upper))
        a=roc_auc_ci(y_test,y_pred_prob)
        print(a)


        
    #PLot every roc from the used classifier per feature selection method 
    a=0
    for k,v in temp_fpr.items():
        plt.plot(v,temp_tpr.get(k),colours[a],label=f"{k}, auc={temp_auc.get(k)}",linewidth=1.5,markersize=1)
        np.savetxt(os.path.join(output_folder,f'{k}_fpr_{data_name}_{hour}.csv'),np.asarray(temp_fpr.get(k)),delimiter=';')
        np.savetxt(os.path.join(output_folder,f'{k}_tpr_{data_name}_{hour}.csv'),np.asarray(temp_tpr.get(k)),delimiter=';')
        a= a+1
    plt.plot(fpr_pim,tpr_pim,'b->',label=f"PIM3Score, auc={auc_pim}",linewidth=1.5,markersize=1)
    plt.legend(loc=4,fontsize='xx-small')
    plt.title(f'{data_name} ROC from {hour} hour data')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    axes = plt.gca()
    axes.set_xlim([0,1])
    axes.set_ylim([0,1])
    fig=plt.gcf()
    pdf_ROC.savefig(fig)
    plt.close(fig)
    count=0

    #PLot calibration from the used classifier per feature selection method
    for k,v in temp_probtrue.items():
        plt.plot(temp_probpred.get(k),v,colours[count],label=f"{k}",linewidth=1.5,markersize=1)
        count += 1
    plt.legend(loc=4,fontsize='xx-small')
    plt.title(f'{data_name} Calibration Curve from {hour} hour data')
    plt.xlabel('Fraction of positives')
    axes = plt.gca()
    axes.set_xlim([0,1])
    axes.set_ylim([0,1])
    fig=plt.gcf()
    pdf_cal.savefig(fig)
    plt.close(fig)
    count=0

    # plot histogram from the used classifier per feature selection method
    for k,v in temp_probtrue.items():
        plt.hist(temp_probpred.get(k),range=(0,1),label=f"{k}",histtype='step',lw=2)
        count += 1
    plt.legend(loc=4,fontsize='xx-small')
    plt.title(f'{data_name} Calibration Curve from {hour} hour data')
    plt.xlabel('Fraction of positives')
    axes = plt.gca()
    axes.set_xlim([0,1])
    axes.set_ylim([0,1])
    fig=plt.gcf()
    pdf_hist.savefig(fig)
    plt.close(fig)


    del temp_fpr,temp_tpr,temp_auc
pdf.close()
pdf_ROC.close()
pdf_cal.close()
pdf_hist.close()



from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve,roc_auc_score
from matplotlib.backends.backend_pdf import PdfPages
# Same functions as before except now specified for Regression algorithms

names = ["Logistic Regression"
        ]

classifiers = [
    LogisticRegression(),
    ]

#x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=1 / 5)

try:
    pdf = PdfPages(os.path.join(output_folder,f"Figures of {hour} hour results_reg.pdf"))
    pdf_ROC= PdfPages(os.path.join(output_folder,f"ROC of {hour} hour results_regression.pdf"))
except PermissionError:
    os.remove(os.path.join(output_folder,f"ROC of {hour} hour results_regression.pdf"))
    os.remove(os.path.join(output_folder,f"Figures of {hour} hour results_reg.pdf"))
    os.remove(os.path.join(output_folder,f"Results_{hour}hours_scores_reg.txt"))
    pdf_ROC= PdfPages(os.path.join(output_folder,f"ROC of {hour} hour results_regression.pdf"))
    pdf = PdfPages(os.path.join(output_folder,f"Figures of {hour} hour results_reg.pdf"))
fpr_pim, tpr_pim, _ = roc_curve(y_test,  x_test['PIM3Score'])
auc_pim = roc_auc_score(y_test, x_test['PIM3Score'])


#x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=1 / 5,random_state=1)

temp_fpr=dict()
temp_tpr=dict()
temp_auc=dict()
for data_name, xd in zip(data_names, x_data):
    # transform columns based on dataframe used
    #columns = xd.columns.tolist()
    x_train_d = x_train#[columns]
    x_test_d = x_test#[columns]
    
    # Add variable for wrapper based feature selection
    if 'Wrap' in data_name:
        wrapper += 1
    else:
        wrapper=0
    
    # create variables for temp storage lateron
    temp_fpr=dict()
    temp_tpr=dict()
    temp_auc=dict()
    temp_probtrue={}
    temp_probpred={}
    temp_score={}

    for name, clf_s in zip(names, classifiers):

        clf,y_pred_clas,y_pred_prob,y_test,x_test_d= machine_learning_function(x_train_d,x_test_d,y_train,y_test,clf_s,wrapper)

        # calculate scoring metrics
        report=classification_report(y_test,y_pred_clas,target_names=['Alive','Death'])
        score=clf.score(x_test_d,y_test)
        average_precision = average_precision_score(y_test, y_pred_prob)
        f1_s=f1_score(y_test, y_pred_clas)

        # write scoring metrics to file
        with open(os.path.join(output_folder,f"Results_{hour}hours_scores_reg.txt"),'a') as file:
            file.write(f"{data_name} with {name} Results for {hour} hours \n\n")
            file.write(f"Classification report \n {report} \n")
            file.write(f"Hold_out_scores {score} \n")
            file.write(f"Average precision score {average_precision} \n")
            file.write(f"F1 score {f1_s} \n\n\n")

        plot_confusion_matrix(clf,x_test_d,y_test)
        plt.title(f"{data_name} with {name} classifier, Results from {hour} hour data")
        fig=plt.gcf()
        pdf.savefig(fig)
        plt.close(fig)

        fpr, tpr, _ = roc_curve(y_test,  y_pred_prob)
        auc = roc_auc_score(y_test, y_pred_prob)


        temp_fpr.update({f'{name}': fpr})
        temp_tpr.update({f'{name}': tpr})
        temp_auc.update({f'{name}': auc})
        print(f"{data_name} with {name} classifier, results from {hour} hour data")
        print("Original ROC area: {:0.3f}".format(roc_auc_score(y_test, y_pred_prob)))
        print("Original ROC area, PIM3: {:0.3f}".format(roc_auc_score(y_test, x_test['PIM3Score'])))
        n_bootstraps = 2000
        rng_seed = 42  # control reproducibility
        bootstrapped_scores = []

        rng = np.random.RandomState(rng_seed)
        for i in range(n_bootstraps):
            # bootstrap by sampling with replacement on the prediction indices
            indices = rng.randint(0, len(y_pred_prob), len(y_pred_prob))
            if len(np.unique(y_test.iloc[indices])) < 2:
                # We need at least one positive and one negative sample for ROC AUC
                # to be defined: reject the sample
                continue

            score = roc_auc_score(y_test.iloc[indices], y_pred_prob[indices])
            bootstrapped_scores.append(score)
            #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

        sorted_scores = np.array(bootstrapped_scores)
        sorted_scores.sort()

        # Computing the lower and upper bound of the 90% confidence interval
        # You can change the bounds percentiles to 0.025 and 0.975 to get
        # a 95% confidence interval instead.
        confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
        confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
        print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
            confidence_lower, confidence_upper))
        a=roc_auc_ci(y_test,y_pred_prob)
        print(a)
    a=0
    for k,v in temp_fpr.items():
        plt.plot(v,temp_tpr.get(k),colours[a],label=f"{k}, auc={temp_auc.get(k)}",linewidth=1.5,markersize=1)
        np.savetxt(os.path.join(output_folder,f'{k}_fpr_{data_name}_{hour}.csv'),np.asarray(temp_fpr.get(k)),delimiter=';')
        np.savetxt(os.path.join(output_folder,f'{k}_tpr_{data_name}_{hour}.csv'),np.asarray(temp_tpr.get(k)),delimiter=';')
        a= a+1
    plt.plot(fpr_pim,tpr_pim,'b->',label=f"PIM3Score, auc={auc_pim}",linewidth=1.5,markersize=1)
    plt.legend(loc=4,fontsize='xx-small')
    plt.title(f'{name} ROC of {hour} hour data')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    axes = plt.gca()
    axes.set_xlim([0,1])
    axes.set_ylim([0,1])
    fig=plt.gcf()
    pdf_ROC.savefig(fig)
    plt.close(fig)

    del temp_fpr,temp_tpr,temp_auc
pdf_ROC.close()
pdf.close()
        


All_data with Random Forest classifier, results from 72 hour data
Original ROC area: 0.900
Original ROC area, PIM3: 0.899
Confidence interval for the score: [0.867 - 0.93]
(0.8526212507023798, 0.9482153576101127)
All_data with Sigmoid SVM classifier, results from 72 hour data
Original ROC area: 0.687
Original ROC area, PIM3: 0.899
Confidence interval for the score: [0.638 - 0.738]
(0.6183564310989509, 0.7565989100943327)
C:\Users\berend\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
All_data with Logistic Re

In [ ]:
for k,v in zip(names,classifiers):
    clf,y_pred_clas,y_pred_prob,y_test,x_test_d,float_columns,cat_list= machine_learning_function(x_train_d,x_test_d,y_train,y_test,clf_s,wrapper)
    try:
        onehot_columns = clf.named_steps['preprocessor'].named_transformers['cat'].named_steps['onehot'].get_feature_names()
        #onehot_columns=clf.get_params()['preprocessor__cat__onehot'].get_feature_names(input_features=cat_list)
        float_columns.extend(onehot_columns)
    except:
        print('hello') 
    try:
        coefs = clf.named_steps["classifier"].coef_.flatten()
    except:
        coefs=clf.named_steps["classifier"].feature_importances_
    import pandas as pd
    # Zip coefficients and names together and make a DataFrame
    zipped = zip(float_columns, coefs)
    df = pd.DataFrame(zipped, columns=["feature", "value"])
    # Sort the features by the absolute value of their coefficient
    df["abs_value"] = df["value"].apply(lambda x: abs(x))
    df["colors"] = df["value"].apply(lambda x: "green" if x > 0 else "red")
    df = df.sort_values("abs_value", ascending=False)


    import seaborn as sns
    fig, ax = plt.subplots(1, 1, figsize=(15, 4))
    sns.barplot(x="feature",
                y="value",
                data=df.head(30),
                palette=df.head(30)["colors"])
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=12)
    ax.set_title(f"Top 20 Features, {data_name},{names},{classifiers}", fontsize=12)
    ax.set_ylabel("Coef", fontsize=12)
    ax.set_xlabel("Feature Name", fontsize=12)

    print(df["feature"].head(20).tolist())